In [ ]:
import numpy as np


def loss(y_true, y_pred):
    return np.sum((y_true - y_pred) ** 2)


def loss_d(y_true, y_pred):
    return 2 * y_pred - 2 * y_true


def sigm_fun(x):
    return 1 / (1 + np.exp(-x))


def sigm_d(sigm_val):
    return sigm_val * (1 - sigm_val)


def relu(x):
    return 0 if x < 0 else x


def relu_d(relu_val):
    return 0 if relu_val <= 0 else 1


def softmax(outputs):
    exp_outputs = [math.exp(o) for o in outputs]
    total = sum(exp_outputs)
    return [eo / total for eo in exp_outputs]


def cross_entropy_loss(y_true, y_pred):
    y_true = int(y_true - 1)
    return -(
        y_true * np.log(y_pred + 1e-10) + (1 - y_true) * np.log(1 - y_pred + 1e-10)
    )


def cross_entropy_loss_d(y_true, y_pred):
    y_true = int(y_true - 1)
    return y_pred - y_true


class Neuron:
    def __init__(self, rng, input_dim, activation_f, activation_f_d):
        self.activation_f = activation_f
        self.activation_f_d = activation_f_d
        weight_list = []
        for i in range(input_dim):
            weight_list.append(rng.uniform(-1, 1))
        self.weights = np.asarray(weight_list)
        self.b = rng.uniform(-1, 1)
        self.b_grad = 0
        self.w_grad = np.zeros(input_dim)
        self.out_grad = 0

    def work(self, inputs):
        self.x = np.array(inputs)
        sum = np.sum(inputs * self.weights, dtype=np.float128) + self.b

        self.output = self.activation_f(sum)
        return self.output

    def x_grad(self, i):
        return self.out_grad * self.activation_f_d(self.output) * self.weights[i]

    def generate_param_grad(self):
        self.w_grad = self.out_grad * self.activation_f_d(self.output) * self.x
        self.b_grad = self.out_grad * self.activation_f_d(self.output)

    def update_weights(self, errors, learning_rate):
        for i in range(len(self.weights)):
            self.weights[i] -= learning_rate * errors[i] * self.x[i]


class NeuralNet:
    def __init__(
        self,
        number_of_neurons_in_layer,
        hidden_layers,
        input_dim,
        number_of_outputs=1,
        seed=10,
        activation_f="sigmoid",
    ):
        if activation_f == "sigmoid":
            act_f = sigm_fun
            act_f_d = sigm_d
        elif activation_f == "relu":
            act_f = relu
            act_f_d = relu_d
        else:
            print("Allowed activation_function values are {'sigmoid', 'relu'}")
            return

        self.rng = np.random.default_rng(seed)
        self.layers = []
        self.neurons: list[Neuron] = []
        layer_1 = [
            Neuron(self.rng, input_dim, act_f, act_f_d)
            for _ in range(number_of_neurons_in_layer)
        ]
        self.layers.append(layer_1)
        self.neurons.extend(layer_1)
        for i in range(hidden_layers - 1):
            current_layer = [
                Neuron(self.rng, number_of_neurons_in_layer, act_f, act_f_d)
                for _ in range(number_of_neurons_in_layer)
            ]
            self.neurons.extend(current_layer)
            self.layers.append(current_layer)
        output_layer = []
        for i in range(number_of_outputs):
            output_neuron = Neuron(
                self.rng,
                number_of_neurons_in_layer,
                activation_f=lambda x: x,
                activation_f_d=lambda x: 1,
            )
            output_layer.append(output_neuron)
            self.neurons.append(output_neuron)
        self.layers.append(output_layer)

    def predict_regre(self, inputs):
        outputs = inputs.copy()
        for layer in self.layers:
            outputs = [neuron.work(outputs) for neuron in layer]
        return outputs[0]

    def predict_class(self, inputs):
        outputs = inputs.copy()
        for layer in self.layers:
            outputs = [neuron.work(outputs) for neuron in layer]
        probabilitie = sigm_fun(outputs[0])

        return probabilitie

    def backprop_single_input(self, input, expected_value):
        y_pred = self.predict_regre(input)
        error = loss(expected_value, y_pred)
        error_d = loss_d(y_pred=y_pred, y_true=expected_value)
        output_neuron = self.layers[-1][0]
        output_neuron.out_grad = error_d
        output_neuron.generate_param_grad()
        next_layer = self.layers[-1]
        for layer in reversed(self.layers[:-1]):
            for i, neuron in enumerate(layer):
                neuron.w_grad = 0
                neuron.out_grad = 0
                neuron.b_grad = 0
                for next_neuron in next_layer:
                    neuron.out_grad += next_neuron.x_grad(i)
                neuron.generate_param_grad()
        return error

    def train_regre(
        self, X, Y, learning_rate=0.05, epochs=50, print_logs=False, batch_size=50
    ):
        n = min(batch_size, len(X))
        for k in range(epochs):
            tot_error = 0
            tot_grad = [(0, 0) for neuron in self.neurons]
            batch_ind = self.rng.integers(low=0, high=len(X), size=n)
            for x, y in zip(X[batch_ind], Y[batch_ind]):
                tot_error += self.backprop_single_input(x, y)
                tot_grad = [
                    (grad[0] + neuron.w_grad, grad[1] + neuron.b_grad)
                    for grad, neuron in zip(tot_grad, self.neurons)
                ]
            tot_error = tot_error / n
            for grad, neuron in zip(tot_grad, self.neurons):
                neuron.weights -= grad[0] / n * learning_rate
                neuron.b -= grad[1] / n * learning_rate
            if print_logs:
                print(f"Epoch: {k}, mean loss: {tot_error}")

    def backprop_binary_input(self, input, expected_value):
        y_pred = self.predict_class(input)
        error = cross_entropy_loss(expected_value, y_pred)
        error_d = cross_entropy_loss_d(y_pred=y_pred, y_true=expected_value)
        output_neuron = self.layers[-1][0]
        output_neuron.out_grad = error_d
        output_neuron.generate_param_grad()
        next_layer = self.layers[-1]
        for layer in reversed(self.layers[:-1]):
            for i, neuron in enumerate(layer):
                neuron.w_grad = 0
                neuron.out_grad = 0
                neuron.b_grad = 0
                for next_neuron in next_layer:
                    neuron.out_grad += next_neuron.x_grad(i)
                neuron.generate_param_grad()
        return error

    def train_class(
        self, X, Y, learning_rate=0.05, epochs=50, print_logs=False, batch_size=50
    ):
        n = min(batch_size, len(X))
        for k in range(epochs):
            tot_error = 0
            tot_grad = [(0, 0) for neuron in self.neurons]
            batch_ind = self.rng.integers(low=0, high=len(X), size=n)
            for x, y in zip(X[batch_ind], Y[batch_ind]):
                tot_error += self.backprop_binary_input(x, y)
                tot_grad = [
                    (grad[0] + neuron.w_grad, grad[1] + neuron.b_grad)
                    for grad, neuron in zip(tot_grad, self.neurons)
                ]
            tot_error = tot_error / n
            for grad, neuron in zip(tot_grad, self.neurons):
                neuron.weights -= grad[0] / n * learning_rate
                neuron.b -= grad[1] / n * learning_rate
            if print_logs:
                print(f"Epoch: {k}, mean loss: {tot_error}")

In [ ]:
import math

def evaluate_model_regre(model, X, Y):
    Y_pred = [model.predict_regre(x) for x in X]
    MSE = sum([(y_pred - y)**2 for y_pred, y in zip(Y_pred,Y)]) / len(Y)
    return math.sqrt(MSE)


def evaluate_model_class(model, X, Y):
    Y_pred = [model.predict_regre(x)  for x in X]
    MSE = sum([(y_pred - y-1)**2 for y_pred, y in zip(Y_pred,Y)]) / len(Y)
    return math.sqrt(MSE)

In [ ]:
def scale(X, min, max):
    return (X.astype(float) - min) / (max - min)

In [ ]:
def rescale(x, min, max):
    return x * (max - min) + min

In [ ]:
train_data = np.genfromtxt('./projekt1/regression/data.activation.train.1000.csv', delimiter=',')
test_data = np.genfromtxt('./projekt1/regression/data.activation.test.1000.csv', delimiter=',')
X_train = train_data[1:, 0]
Y_train = train_data[1:, 1]
X_test = test_data[1:, 0]
Y_test = test_data[1:, 1]

net = NeuralNet(hidden_layers=3,number_of_neurons_in_layer=30,input_dim=1, activation_f='relu')
net.train_regre(X_train, Y_train, learning_rate=0.01, epochs=1000, print_logs=True)

print(f"RMSE on train data {evaluate_model_regre(net, X_train, Y_train)}")
print(f"RMSE on test data {evaluate_model_regre(net, X_test, Y_test)}")

In [ ]:
train_data = np.genfromtxt(
    "./projekt1/regression/data.activation.train.1000.csv", delimiter=","
)
test_data = np.genfromtxt(
    "./projekt1/regression/data.activation.test.1000.csv", delimiter=","
)
X_train = train_data[1:, 0]
Y_train = train_data[1:, 1]
X_test = test_data[1:, 0]
Y_test = test_data[1:, 1]
minx = min(X_train)
maxx = max(X_train)

net = NeuralNet(
    hidden_layers=3, number_of_neurons_in_layer=30, input_dim=1, activation_f="relu"
)
X_scaled = scale(X_train, minx, maxx)
X_test_scaled = scale(X_test, minx, maxx)
net.train_regre(X_scaled, Y_train, learning_rate=0.0001, epochs=500, print_logs=True)

print(f"RMSE on train data {evaluate_model_regre(net, X_scaled, Y_train)}")
print(f"RMSE on test data {evaluate_model_regre(net, X_test_scaled, Y_test)}")

Plot scaled

In [ ]:

import matplotlib.pyplot as plt
ind = np.argsort(X_test_scaled)
plt.plot(X_test_scaled, Y_test, label = 'test')
Y_pred = [net.predict_regre(x) for x in X_test_scaled]
plt.plot(X_test_scaled, Y_pred, label = 'pred')
plt.legend()

Plot not scaled


In [ ]:
import matplotlib.pyplot as plt
ind = np.argsort(X_train)
plt.plot(X_test, Y_test, label = 'test')
Y_pred = [net.predict_regre(x) for x in X_test]
plt.plot(X_test, Y_pred, label = 'pred')
plt.legend()

# Classifcation

In [ ]:
train_data = np.genfromtxt('./projekt1/classification/data.simple.train.1000.csv', delimiter=',')
test_data = np.genfromtxt('./projekt1/classification/data.simple.test.1000.csv', delimiter=',')
X_train = train_data[1:, :2]
Y_train = train_data[1:, 2]
X_test = test_data[1:, :2]
Y_test = test_data[1:, 2]

net = NeuralNet(hidden_layers=3,number_of_neurons_in_layer=30,input_dim=2, number_of_outputs = 1)
net.train_class(X_train, Y_train, learning_rate=0.01, epochs=1000, print_logs=True)

print(f"RMSE on train data {evaluate_model_class(net, X_train, Y_train)}")
print(f"RMSE on test data {evaluate_model_class(net, X_test, Y_test)}")

In [ ]:
import seaborn as sns
X_test = test_data[1:, 0]
Y_test = test_data[1:, 1]
C_test = test_data[1:, 2]

# Create a scatter plot using Seaborn
plt.figure(figsize=(10, 6))

# Use scatterplot with NumPy arrays
Y_pred = [0 if net.predict_class(input) < 0.5 else 1 for input in test_data[1:, :2]]
sns.scatterplot(x=X_test, y=Y_test, hue=Y_pred, palette='viridis', s=100, edgecolor='k', alpha=0.7)

# Add labels and title
plt.xlabel('X')
plt.ylabel('Y')
plt.legend(title='Klasa')
plt.grid(True)

# Wpływ liczby warstw

In [ ]:
train_data = np.genfromtxt(
    "./projekt1/regression/data.activation.train.1000.csv", delimiter=","
)
test_data = np.genfromtxt(
    "./projekt1/regression/data.activation.test.1000.csv", delimiter=","
)
X_train = train_data[1:, 0]
Y_train = train_data[1:, 1]
X_test = test_data[1:, 0]
Y_test = test_data[1:, 1]
minx = min(X_train)
maxx = max(X_train)

numbers_of_layer = [0,1,2,3,4]
numbers_of_neurons = [1,5,20,50,100]

nets = []
for l in numbers_of_layer:
    for n in numbers_of_neurons: 
        nets.append(NeuralNet(hidden_layers=l, number_of_neurons_in_layer=n, input_dim=1, activation_f="relu"))
    

X_scaled = scale(X_train, minx, maxx)
X_test_scaled = scale(X_test, minx, maxx)
for net in nets:
    net.train_regre(X_scaled, Y_train, learning_rate=0.0001, epochs=500, print_logs=True)
    print("Done")


In [ ]:

import matplotlib.pyplot as plt

for net in nets:
    plt.plot(X_test_scaled, Y_test, label = 'test')
    Y_pred = [net.predict_regre(x) for x in X_test_scaled]
    plt.plot(X_test_scaled, Y_pred, label = 'pred')
    plt.show()
             



In [ ]:
train_data = np.genfromtxt(
    "./projekt1/regression/data.activation.train.1000.csv", delimiter=","
)
test_data = np.genfromtxt(
    "./projekt1/regression/data.activation.test.1000.csv", delimiter=","
)
X_train = train_data[1:, 0]
Y_train = train_data[1:, 1]
X_test = test_data[1:, 0]
Y_test = test_data[1:, 1]
minx = min(X_train)
maxx = max(X_train)

numbers_of_layer = [0,1,2,3,4]
numbers_of_neurons = [1,5,20,50,100]

net = NeuralNet(hidden_layers=4, number_of_neurons_in_layer=100, input_dim=1, activation_f="relu")
    

X_scaled = scale(X_train, minx, maxx)
X_test_scaled = scale(X_test, minx, maxx)
net.train_regre(X_scaled, Y_train, learning_rate=0.0001, epochs=500, print_logs=True)


In [ ]:
import matplotlib.pyplot as plt

plt.plot(X_test_scaled, Y_test, label = 'test')
Y_pred = [net.predict_regre(x) for x in X_test_scaled]
plt.plot(X_test_scaled, Y_pred, label = 'pred')
plt.show()             